In [21]:
import os
import numpy as np
from glob import glob
import DMDana.lib.DMDparser as DMDparser
import importlib

In [22]:
folders='''/scratch/groups/ping_group/zbai29/RhSi/DMD/0.2eV-Andrew/0.0005
/scratch/groups/ping_group/zbai29/RhSi/DMD/0.2-init2/0.0005
/scratch/groups/ping_group/zbai29/RhSi/DMD/0.2eV-Andrew/low-window/0.0005
/scratch/groups/ping_group/zbai29/RhSi/DMD/0.2eV-Andrew/low-window/highK/0.0005
/scratch/groups/ping_group/zbai29/RhSi/DMD/0.3eV/0.00005-Andrew
/scratch/groups/ping_group/zbai29/RhSi/DMD/0.3eV/0.00050
/scratch/groups/ping_group/zbai29/RhSi/DMD/0.5eV-Andrew/0.00005
/scratch/groups/ping_group/zbai29/RhSi/DMD/Rafi-LC/0.00050
/scratch/groups/ping_group/zbai29/RhSi/DMD/0.6eV-Andrew/0.00005
/scratch/groups/ping_group/zbai29/RhSi/DMD/0.7eV/0.00050
/scratch/groups/ping_group/zbai29/RhSi/DMD/0.8eV/0.00050
/scratch/groups/ping_group/zbai29/RhSi/init/init_N1_W0.381/DMD/0.5eV_with_ee/0.00005
/scratch/groups/ping_group/zbai29/RhSi/init/init_N1_W0.381/DMD/0.5eV/0.00005
/scratch/groups/ping_group/zbai29/RhSi/init/init_N1_W0.381/DMD/0.5eV_with_ee_epsilon1/0.00005
/scratch/groups/ping_group/zbai29/RhSi/init/init_N1_W0.381/DMD/0.5eV-RTA-tau0.011/0.00005
/scratch/groups/ping_group/zbai29/RhSi/init/init_N1_W0.381/DMD/0.5eV-RTA-tau0.11/0.00005
/scratch/groups/ping_group/zbai29/RhSi/init/init_N1_W0.381/DMD/0.5eV-RTA-tau1.1/0.00005
/scratch/groups/ping_group/zbai29/RhSi/init/init_N1_W0.381/DMD/0.5eV/0.00005
/scratch/groups/ping_group/zbai29/RhSi/init/init_morek/DMD/0.5eV/0.00005
/scratch/groups/ping_group/zbai29/RhSi/init/init_morek/DMD/0.5eV/0.00005
/scratch/groups/ping_group/zbai29/RhSi/init/init_morek_bigsmearing/DMD/0.5eV/0.00005
/scratch/groups/ping_group/zbai29/RhSi/init/init_N1_W0.381/DMD/0.5eV-linear/0.00005
/scratch/groups/ping_group/zbai29/GaAs_SOC_20231130/ld-dmd-current/t300/mult8_gauss0.005/ExEy_Diag_Occup_45
/scratch/groups/ping_group/zbai29/GaAs_SOC_20231130/ld-dmd-current/t300/mult8_gauss0.005/ExEy_Diag_Occup_45_pheno
/scratch/groups/ping_group/zbai29/GaAs_SOC_20231130/ld-dmd-current/t300/mult8_gauss0.005/ExEy_Diag_Occup_45_pheno_lightreverse
/scratch/groups/ping_group/zbai29/GaAs_SOC_20231130/ld-dmd-current/t300/mult8_gauss0.005/DMD_new/1.5eV/0.00050-RTA-nondoped
/scratch/groups/ping_group/zbai29/GaAs_SOC_20231130/ld-dmd-current/t300/mult8_gauss0.005/DMD_new/1.5eV/0.00050-RTA
/scratch/groups/ping_group/zbai29/GaAs_SOC_20231130/ld-dmd-current/t300/mult8_gauss0.005/DMD_new/1.5eV/0.00050-RTA0.011
/scratch/groups/ping_group/zbai29/GaAs_SOC_20231130/ld-dmd-current/t300/mult8_gauss0.005/DMD_new/1.5eV/0.00050-rec0.22-RTA
/scratch/groups/ping_group/zbai29/GaAs_SOC_20231130/ld-dmd-current/t300/mult8_gauss0.005/DMD_new/1.5eV/0.00050-rec0.022-RTA
/scratch/groups/ping_group/zbai29/GaAs_SOC_20231130/ld-dmd-current/t300/mult8_gauss0.005/DMD_new/1.5eV/0.00050-noee
/scratch/groups/ping_group/zbai29/GaAs_SOC_20231130/ld-dmd-current/t300/mult8_gauss0.005/DMD_new/1.5eV/0.00050-eph-nondoped-rec0.22
/scratch/groups/ping_group/zbai29/GaAs_SOC_20231130/ld-dmd-current/t300/mult8_gauss0.005/DMD_new/1.5eV/0.00050-ephee-nondoped-rec0.22'''

In [23]:

from DMDana.lib.constant import *
normal_filename='/occupations-[0-9][0-9][0-9][0-9][0-9].out'
def parse_occup(occup_path):
    t=DMDparser.read_text_from_file(occup_path,['nk ='],[12],True,float)[0]
    assert t!=None, "file %s might be empty"%occup_path
    nstep=occup_path.split('/')[-1].split('_')[-1].split('-')[-1].split('.')[0]
    if nstep!='t0':
        nstep=int(nstep)
    else:
        nstep=1
    return nstep,t
def get_tstep(folder):
    DMDparam_value=DMDparser.get_DMD_param(folder)

    return float(DMDparam_value['tstep_pump'])*fs

Check if these is any filename mistake

In [24]:
print('excel_i\tnum\ttime\tstep_fs\ttime_match')
for i,folder in enumerate(folders.split()):
    excel_i=2+i
    t0=glob(folder+'/occupations_t0.out')
    #assert t0!=[], "Did not found occupations_t0.out at folder %s"%folder
    not0 = sorted(glob(folder+normal_filename))
    witht0=t0+not0
    tstep=get_tstep(folder)
    
    for occup_path in witht0:
        nstep, t = parse_occup(occup_path)
        #print(t,nstep)
        realstepnum=round(t/tstep)
        realtime=t/fs
        timematch=(nstep==realstepnum)
        if not timematch:
            print('%d\t%d\t%.1f\t%.3f\t%s\t%s'%(excel_i,len(witht0),realtime,(tstep/fs),timematch,folder))
            break
    if timematch:
        print('%d\t%d\t%.1f\t%.3f\t%s\t%s'%(excel_i,len(witht0),realtime,(tstep/fs),timematch,folder))


excel_i	num	time	step_fs	time_match
2	1	1.0	1.000	True	/scratch/groups/ping_group/zbai29/RhSi/DMD/0.2eV-Andrew/0.0005
3	1	1.0	1.000	True	/scratch/groups/ping_group/zbai29/RhSi/DMD/0.2-init2/0.0005
4	1	1.0	1.000	True	/scratch/groups/ping_group/zbai29/RhSi/DMD/0.2eV-Andrew/low-window/0.0005
5	1	2751.0	1.000	True	/scratch/groups/ping_group/zbai29/RhSi/DMD/0.2eV-Andrew/low-window/highK/0.0005
6	1	1.0	1.000	True	/scratch/groups/ping_group/zbai29/RhSi/DMD/0.3eV/0.00005-Andrew
7	1	1.0	1.000	True	/scratch/groups/ping_group/zbai29/RhSi/DMD/0.3eV/0.00050
8	1	1.0	1.000	True	/scratch/groups/ping_group/zbai29/RhSi/DMD/0.5eV-Andrew/0.00005
9	1	1.0	1.000	True	/scratch/groups/ping_group/zbai29/RhSi/DMD/Rafi-LC/0.00050
10	1	1.0	1.000	True	/scratch/groups/ping_group/zbai29/RhSi/DMD/0.6eV-Andrew/0.00005
11	1	1.0	1.000	True	/scratch/groups/ping_group/zbai29/RhSi/DMD/0.7eV/0.00050
12	1	14771.0	1.000	True	/scratch/groups/ping_group/zbai29/RhSi/DMD/0.8eV/0.00050
13	176	17500.0	1.000	True	/scratch/groups/ping

Correct filenames

In [25]:
import shutil
def correct_occup_name(folder):
    t0=glob(folder+'/occupations_t0.out')
    #assert t0!=[], "Did not found occupations_t0.out at folder %s"%folder
    not0 = sorted(glob(folder+normal_filename))
    witht0=t0+not0
    for occup_path in witht0:
        nstep, t =parse_occup(occup_path)
        tstep=get_tstep(folder)
        realstep=round(t/tstep)
        if realstep!=nstep:
            backupname=occup_path.replace('occupations','occupations_bk')
            newname='/'.join(occup_path.split('/')[:-1]+['occupations-%.5d.out'%realstep])
            print(occup_path,backupname,newname)
            os.rename(occup_path,backupname)
            shutil.copy(backupname,newname)

In [26]:
correct_occup_name('/scratch/groups/ping_group/zbai29/RhSi/DMD/0.2eV-Andrew/low-window/highK/0.0005')

Correct some naming mistake due to some previous bug of this scirpt

In [27]:
for folder in folders.split():
    ls=glob(folder+normal_filename.replace('-','_'))
    if len(ls)!=0:
        print(folder)
    for path in ls:
        os.rename(path,path.replace('occupations_','occupations-'))
